# 環保署各測站的 PM2.5 資料

這個 jupyter notebook 提供了工具和範例，把環保署測站資料的 PM2.5 集合在一起，每個欄位是一個測站的時間序列。在使用之前，請先下載 2015 年環保署所有測站資料，並解壓縮放在 `../data/104_HOUR_00_20160323` 中。

首先，我們先載入需要用到的函式庫，以及在「資料清理」課程中使用到的一些函數：

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re, os

def detect_epa_nan(x):
    ''' Search for missing value symbol and assign np.nan '''
    if re.findall('\#|\*|x', str(x))!=[]:
        return(np.nan)
    else:
        return(x)

def detect_epa_norain(x):
    ''' Replace 'NR' (no-rain) with 0 '''
    if str(x)=='NR':
        return(0)
    else:
        return(x)

def clean_epa_station(x):
    ''' Clean up a EPA station dataset '''
    # Rename columns
    col_names = ['date','station','item','h00','h01','h02','h03','h04','h05','h06','h07','h08','h09',
                'h10','h11','h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
    x.columns = col_names
    # Process NA and NR
    floatdata = x.iloc[:,3:]
    floatdata = floatdata.applymap(detect_epa_nan)
    floatdata = floatdata.applymap(detect_epa_norain)
    floatdata.astype(np.float32)
    x.iloc[:,3:] = floatdata
    # Done
    return(x)

# Retrieve one item from EPA data and form a time series
def retrieve_epa_item(data, var):
    tmp = data.loc[data['item']==var,:]
    ts = pd.melt(tmp, id_vars=['date'], value_vars=tmp.keys()[3:], var_name='hour', value_name=var)
    ts[var] = ts[var].astype(np.float32)
    return(ts)


## 尋找目錄下的所有檔案

由於環保署資料包含了很多個檔案，又依照空品區的劃分，放在不同的資料夾底下，因此我們需要借助 python 的 [`os.walk()`](https://docs.python.org/3/library/os.html) 函數，幫我們自動「走遍」資料夾底下的每個檔案。

此外，由於資料的檔名包含了測站名稱，我們可以順便把測站名稱取出來，作為欄位的名稱，所以我們需要用到 [`str.find()`](https://www.tutorialspoint.com/python/string_find.htm) 函數，來幫我們找到測站名稱在檔名裡的位置。

In [2]:
# Walk through the specified path and find all files ended with 'xls'
def find_xls_files(path):
    ids = []    # 測站名稱
    urls = []   # 檔案完整路徑
    for root, dirs, files in os.walk(path):             # os.walk() 會傳回「根目錄」、「路徑」、和「檔名」三個字串 list
        for fname in files:                             # 每個檔名
            if(fname.endswith(".xls")):                 # 如果是以 xls 結尾
                fid = fname[(fname.find('104年')+4):][:(fname.find('_2016')-5)]
                ids.append(fid)
                urls.append(os.path.join(root, fname))
    return((ids,urls))


由於環保署資料的檔名是以 *YYY年XX站_YYYYMMDD.xls* 的格式命名 ，例如：104年花蓮站_20160320.xls，我們要取出字串裡的 *XX*，所以我們要：

1. 從完整檔名裡找到 *'104年'* 之後的字串 A
2. 從 A 裡找到 *'_2016'* 前面的字串，不包含「站」

因此，我們的寫法會是：

In [3]:
fn = '104年花蓮站_20160320.xls'
fn1 = fn[(fn.find('104年')+4):]
print(fn1)
fn2 = fn1[:(fn1.find('_2016')-1)]
print(fn2)

花蓮站_20160320.xls
花蓮


In [4]:
ids, epafiles = find_xls_files('../data/104_HOUR_00_20160323/')
print(ids)

['二林', '南投', '埔里', '大里', '彰化', '忠明', '沙鹿', '竹山', '線西', '西屯', '豐原', '三重', '中壢', '中山', '古亭', '土城', '基隆', '士林', '大同', '大園', '平鎮', '新店', '新莊', '松山', '板橋', '林口', '桃園', '永和', '汐止', '淡水', '菜寮', '萬華', '萬里', '觀音', '陽明', '龍潭', '冬山', '宜蘭', '三義', '新竹', '湖口', '竹東', '苗栗', '頭份', '臺東', '花蓮', '關山', '金門', '馬公', '馬祖', '善化', '嘉義', '安南', '崙背', '斗六', '新港', '新營', '朴子', '臺南', '臺西', '麥寮', '仁武', '前金', '前鎮', '大寮', '小港', '屏東', '左營', '復興', '恆春', '林園', '楠梓', '橋頭', '潮州', '美濃', '鳳山']


## 實際進行資料的讀取、清理與合併

接下來，我們就用上面的函數，讀進各個測站的資料，清理之後，抽出 PM2.5 的資料，然後用日期跟時間的欄位加以合併。

In [5]:
# Collect data from all files
f = epafiles[0]                                         # Start from the first file
tmp = pd.read_excel(f)                                  # Read in file
tmp = clean_epa_station(tmp)                            # Clean up nan and no-rain
data = retrieve_epa_item(tmp, 'PM2.5')                  # Retrieve PM2.5
nancounts = []
for f in epafiles[1:]:
    tmp = pd.read_excel(f)                                  # Read in file
    tmp = clean_epa_station(tmp)                            # Clean up nan and no-rain
    tmp = retrieve_epa_item(tmp, 'PM2.5')                   # Retrieve PM2.5
    data = data.merge(tmp, on=['date','hour'], how='left')  # Aggregate data
    nancounts.append(tmp.apply(lambda x: len(x)-x.count())) # Count the number of nan
    #print(f + ": " + str(len(tmp)))

data = data.sort_values(['date', 'hour'])                   # Sort data
cnames = list(data.columns[:2]) + ids
data.columns = cnames
data.head()

,date,hour,二林,南投,埔里,大里,彰化,忠明,沙鹿,竹山,...,屏東,左營,復興,恆春,林園,楠梓,橋頭,潮州,美濃,鳳山
0,2015/01/01,h00,29.0,48.0,55.0,53.0,36.0,44.0,15.0,51.0,...,77.0,77.0,86.0,21.0,71.0,76.0,83.0,61.0,37.0,72.0
365,2015/01/01,h01,30.0,44.0,51.0,55.0,39.0,37.0,21.0,48.0,...,64.0,86.0,74.0,17.0,77.0,74.0,67.0,69.0,39.0,62.0
730,2015/01/01,h02,41.0,39.0,47.0,58.0,36.0,38.0,37.0,46.0,...,61.0,83.0,58.0,19.0,63.0,78.0,66.0,71.0,39.0,51.0
1095,2015/01/01,h03,46.0,37.0,40.0,53.0,33.0,22.0,56.0,40.0,...,46.0,71.0,54.0,15.0,59.0,66.0,56.0,65.0,36.0,37.0
1460,2015/01/01,h04,69.0,34.0,32.0,43.0,36.0,29.0,67.0,34.0,...,43.0,66.0,57.0,21.0,56.0,46.0,54.0,47.0,37.0,44.0


## 資料的輸出

Python 的輸出/輸入非常多樣，我們這裡介紹 pandas.DataFrame 內建的輸出/輸入工具。在我們讀取資料的時候，使用了 [pandas.read_csv()](https://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table) 與 [pandas.read_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)，想當然耳，pandas 也提供了輸出成這兩種格式的工具 [pandas.DataFrame.to_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html) / [pandas.DataFrame.to_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html)。

對於「表格式」的資料來說，CSV 大概是最常用的資料格式，下面我們就把整理好的資料輸出成 csv 檔，方便後續的分析來使用。

In [ ]:
data.to_csv('../data/pm25_2015.csv', index=False)